# Import my personal printing object

In [2]:
from parameters import ParameterContainer

# Import and name constants

In [3]:
import numpy as np
from scipy import constants
c = constants.physical_constants["speed of light in vacuum"][0] #m/sec by default
e = constants.physical_constants["elementary charge"][0] #m/sec by default
m_e_c2 = constants.physical_constants["electron mass energy equivalent in MeV"][0]*10**-3 #in GeV
r_e = constants.physical_constants["classical electron radius"][0]#in m
h_bar = constants.physical_constants["Planck constant over 2 pi in eV s"][0] #in eV s

# Name and assign input parameters

In [4]:
electron_energy = 1. #energy of the beam in GeV

# Calculate and store derived parameters

In [5]:
electron_kinetic_energy = (electron_energy - m_e_c2)*10**9 #in eV
electron_momentum = np.sqrt(electron_energy**2 - m_e_c2**2)*10**9/c #in eV/c
electron_gamma = electron_kinetic_energy/(m_e_c2*10**9) + 1

# Ring/turn parameters storing and reporting

In [6]:
quad_B = 1. #magnetic field in the turns in T

ring_parameters = ParameterContainer()
ring_parameters.addParameter("electron energy",electron_energy,"GeV")
ring_parameters.addParameter("turn quadrupole B",quad_B,"T")
rho = ring_parameters.addParameter("rho",electron_momentum/quad_B,"m").value()
L_bend = ring_parameters.addParameter("length of bend",np.pi/2.*rho,"m")
U0 = ring_parameters.addParameter("synchrotron energy loss per turn",4.*np.pi/3.*r_e*electron_gamma**4/rho*m_e_c2*10**6,"keV")
E_photon = ring_parameters.addParameter("energy of sychrotron emitted photon",h_bar*3./2.*c*electron_gamma**3/rho/1000,"keV")

print(str(ring_parameters))


electron energy = 1.0 GeV
turn quadrupole B = 1.0 T
rho = 3.33564051648 m
length of bend = 5.2396118708 m
synchrotron energy loss per turn = 26.5204657704 keV
energy of sychrotron emitted photon = 0.665025765843 keV


# Doublet parameters storing and reporting 

In [7]:
G1 = 50. #Magnetic field gradient for the first doublet quad in T/m.
G2 = 100. #Magnetic field gradient for the second doublet quad in T/m.
L_quad = 0.10 #Length of the doublet quads in m.

doublet_parameters = ParameterContainer()
doublet_parameters.addParameter("Quad length",L_quad,"m")
doublet_parameters.addParameter("Quad gradient 1",G1,"T/m")
doublet_parameters.addParameter("Quad gradient 2",G2,"T/m")
K1 = doublet_parameters.addParameter("K1",G1/electron_momentum,"1/m^2").value()
K2 = doublet_parameters.addParameter("K2",G2/electron_momentum,"1/m^2").value()
f1 = doublet_parameters.addParameter("f1",electron_momentum/(G1*L_quad),"m").value()
f2 = doublet_parameters.addParameter("f2",electron_momentum/(G2*L_quad),"m").value()
d = doublet_parameters.addParameter("Post quad drift",np.sqrt(f2**2 * f1 / (f1-f2)),"m").value()
quad_spacing = doublet_parameters.addParameter("Quad spacing",f2 * f1 / d,"m").value()
doublet_parameters.addParameter("Doublet focus",d + quad_spacing,"m")

print str(doublet_parameters)

Quad length = 0.1 m
Quad gradient 1 = 50.0 T/m
Quad gradient 2 = 100.0 T/m
K1 = 14.989624857 1/m^2
K2 = 29.9792497141 1/m^2
f1 = 0.667128103296 m
f2 = 0.333564051648 m
Post quad drift = 0.471730805761 m
Quad spacing = 0.471730805761 m
Doublet focus = 0.943461611522 m


# Laser parameters

In [78]:
laser_parameters = ParameterContainer()
laser_wavelength = laser_parameters.addParameter("wavelength",780,"nm").value()
a_0 = laser_parameters.addParameter("a_0",2.1, "").value()
intensity = laser_parameters.addParameter("intensity",a_0**2/(laser_wavelength*1e-3)**2/7.3e-19,"W/cm^2").value()

print laser_parameters

wavelength = 780 nm
a_0 = 2.1 
intensity = 9.92948042474e+18 W/cm^2


# Plasma parameters

In [76]:
plasma_parameters = ParameterContainer()
n = plasma_parameters.addParameter("plasma density",3e17,"cm^-3").value()
E_max = plasma_parameters.addParameter("Max E field",np.sqrt(n * a_0**3)/1e9,"GV/cm").value()
plasma_wavelength = plasma_parameters.addParameter("plasma wavelength",0.1*np.sqrt(1e17/n),"mm").value()
w_pe = plasma_parameters.addParameter("plasma frequency",2.*np.pi*c/(plasma_wavelength*1e-3),"rad/sec").value()
R_b = plasma_parameters.addParameter("bubble size",2*np.sqrt(a_0)*c/w_pe,"m").value()
laser_power = plasma_parameters.addParameter("total laser power",np.pi*R_b**2*intensity*100.**2/1e12,"TW").value()
L_p = plasma_parameters.addParameter("depletion length",0.5 / a_0 * plasma_wavelength**3 / (laser_wavelength*1e-6)**2/10,"cm").value()
L_d = plasma_parameters.addParameter("dephasing length",L_p*a_0/10,"cm").value()
max_energy = plasma_parameters.addParameter("maximum energy possible",E_max*np.min([L_p,L_d]),"GeV").value()
total_plasma_length = plasma_parameters.addParameter("total plasma length",3. / E_max,"cm").value()

print plasma_parameters

plasma density = 3e+17 cm^-3
Max E field = 2.16506350946 GV/cm
plasma wavelength = 0.057735026919 mm
plasma frequency = 3.26258021834e+13 rad/sec
bubble size = 2.90575841566e-05 m
total laser power = 373.281814988 TW
depletion length = 6.32643292998 cm
dephasing length = 1.58160823249 cm
maximum energy possible = 3.42428227044 GeV
total plasma length = 1.38564064606 cm


In [80]:
plasma_parameters_2 = ParameterContainer()
n = plasma_parameters_2.addParameter("plasma density",1.75e17,"cm^-3").value()
E_max = plasma_parameters_2.addParameter("Max E field",np.sqrt(n * a_0**2)/1e9,"GV/cm").value()
plasma_wavelength = plasma_parameters_2.addParameter("plasma wavelength",0.1*np.sqrt(1e17/n),"mm").value()
w_pe = plasma_parameters_2.addParameter("plasma frequency",2.*np.pi*c/(plasma_wavelength*1e-3),"rad/sec").value()
R_b = plasma_parameters_2.addParameter("bubble size",2*np.sqrt(a_0)*c/w_pe,"m").value()
laser_power = plasma_parameters_2.addParameter("total laser power",np.pi*R_b**2*intensity*100.**2/1e12,"TW").value()
L_p = plasma_parameters_2.addParameter("depletion length",0.5 / a_0 * plasma_wavelength**3 / (laser_wavelength*1e-6)**2/10,"cm").value()
L_d = plasma_parameters_2.addParameter("dephasing length",L_p*a_0/10,"cm").value()
max_energy = plasma_parameters_2.addParameter("maximum energy possible",E_max*np.min([L_p,L_d]),"GeV").value()
total_plasma_length = plasma_parameters_2.addParameter("total plasma length",3. / E_max,"cm").value()
critical_power = total_plasma_length = plasma_parameters_2.addParameter("critical power",21.5*( a_0*R_b/(laser_wavelength*1e-9) )**2/1000,"TW").value()

print plasma_parameters_2

plasma density = 1.75e+17 cm^-3
Max E field = 0.878493027861 GV/cm
plasma wavelength = 0.0755928946018 mm
plasma frequency = 2.4918368019e+13 rad/sec
bubble size = 3.4869100988e-05 m
total laser power = 379.278214064 TW
depletion length = 16.9045817963 cm
dephasing length = 3.54996217723 cm
maximum energy possible = 3.11861702186 GeV
total plasma length = 3.41493888381 cm
critical power = 189.482604084 TW
